In [1]:
# imports
from pickle import TRUE
import pandas as pd
import requests
import time
import warnings
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import base64
from io import StringIO
import creds
import numpy as np


warnings.filterwarnings("ignore")

In [2]:

# Function to push DataFrame to GitHub
def push_dataframe_to_github(dataframe, file_name, repository):
    # Convert DataFrame to CSV in-memory
    csv_data = StringIO()
    dataframe.to_csv(csv_data, index=False)
    csv_content = csv_data.getvalue()

    # Encode content to Base64
    encoded_content = base64.b64encode(csv_content.encode()).decode()

    # Check if the file exists
    url_get = f'https://api.github.com/repos/{username}/{repository}/contents/{file_name}'
    headers_get = {'Authorization': f'token {creds.token}'}

    response_get = requests.get(url_get, headers=headers_get)
    response_get_json = response_get.json()

    # Extract the SHA from the response
    current_sha = response_get_json.get('sha')

    # Push data directly to GitHub using GitHub API
    url_put = f'https://api.github.com/repos/{username}/{repository}/contents/{file_name}'
    headers_put = {
        'Authorization': f'token {creds.token}',
        'Content-Type': 'application/json'
    }

    data_put = {
        'message': f'Update {file_name}',
        'content': encoded_content
    }

    # If the file exists, update it; otherwise, create it
    if current_sha is not None:
        data_put['sha'] = current_sha

    response_put = requests.put(url_put, headers=headers_put, json=data_put)

    if response_put.status_code == 200:
        print(f'Data pushed to {file_name} on GitHub successfully!')
    elif response_put.status_code == 201:
        print(f'{file_name} created on GitHub successfully!')
    else:
        print(f'Error: {response_put.status_code}, {response_put.text}')

# GitHub repository details
username = 'alanhassan'
repository = 'stocks_panel'

## df_nome

In [3]:
service=Service(ChromeDriverManager().install())

# Instanciando o Objeto ChromeOptions
options = webdriver.ChromeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Chrome(service=service, options=options)

In [4]:
wd_Chrome.get("https://www.fundamentus.com.br/")

wd_Chrome.find_element('xpath', '/html/body/div[1]/div[1]/form/fieldset/input[2]').click()

table_element = wd_Chrome.find_element('id', 'test1')

# Get the HTML content of the table
table_html = table_element.get_attribute('outerHTML')

# Use Pandas to read the HTML and create a DataFrame
df_nome = pd.read_html(table_html)[0]

# Close the browser
wd_Chrome.quit()

## df_setor

In [5]:
service=Service(ChromeDriverManager().install())

# Instanciando o Objeto ChromeOptions
options = webdriver.ChromeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Chrome(service=service, options=options)

wd_Chrome.get("https://www.fundamentus.com.br/")


wd_Chrome.find_element('xpath', '/html/body/div[1]/div[1]/div[1]/span/a[1]').click()

In [6]:
# Find the dropdown element by its name attribute
dropdown_element = wd_Chrome.find_element('name', 'setor')

# Create a Select object to interact with the dropdown
select = Select(dropdown_element)

# Get all available options in the dropdown
all_options = select.options

# Get the total number of options
num_options = len(all_options)

In [7]:
# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['Papel', 'Setor'])

for i in range(1, num_options):
    wd_Chrome.get("https://www.fundamentus.com.br/")

    wd_Chrome.find_element('xpath', '/html/body/div[1]/div[1]/div[1]/span/a[1]').click()

    dropdown_element = wd_Chrome.find_element('name', 'setor')

    select = Select(dropdown_element)

    select.select_by_index(i)

    setor = select.first_selected_option.text

    wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/form/input').click()

    table_element = wd_Chrome.find_element('id', 'resultado')

    table_html = table_element.get_attribute('outerHTML')

    df = pd.read_html(table_html)[0]

    df_setor = df[['Papel']].copy()
    df_setor['Setor'] = setor

    # Append the current iteration's DataFrame to the result DataFrame
    result_df = pd.concat([result_df, df_setor], ignore_index=True)

    print(i)

    time.sleep(2)

# Close the browser
wd_Chrome.quit()

df_setor = result_df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


## df

In [8]:
service=Service(ChromeDriverManager().install())

# Instanciando o Objeto ChromeOptions
options = webdriver.ChromeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Chrome(service=service, options=options)

wd_Chrome.get("https://www.fundamentus.com.br/")


wd_Chrome.find_element('xpath', '/html/body/div[1]/div[1]/div[1]/span/a[1]').click()

wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/form/input').click()

table_element = wd_Chrome.find_element('id', 'resultado')

# Get the HTML content of the table
table_html = table_element.get_attribute('outerHTML')

# Use Pandas to read the HTML and create a DataFrame
df = pd.read_html(table_html)[0]

# Close the browser
wd_Chrome.quit()

In [9]:
# Assuming df is your DataFrame
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x).replace(',', '.'))

# adjusting types

df['P/L'] = df['P/L'].str.replace('[.,]', '', regex=True).astype(int)
df['EV/EBITDA'] = df['EV/EBITDA'].str.replace('[.,]', '', regex=True).astype(int)
df['P/VP'] = df['P/VP'].str.replace('[.,]', '', regex=True).astype(int)
df['ROE'] = df['ROE'].str.replace('[.,%]', '', regex=True).astype(int)
df['Div.Yield'] = df['Div.Yield'].str.replace('[.,%]', '', regex=True).astype(int)
df['Cresc. Rec.5a'] = df['Cresc. Rec.5a'].str.replace('[.,%]', '', regex=True).astype(int)
df['Patrim. Líq'] = df['Patrim. Líq'].str.replace('[.,]', '', regex=True).astype(np.int64)

# adjusting columns

df['Cotação'] = df['Cotação']/100
df['P/L'] = df['P/L']/100
df['EV/EBITDA'] = df['EV/EBITDA']/100
df['P/VP'] = df['P/VP']/100
df['ROE'] = df['ROE']/100
df['Div.Yield'] = df['Div.Yield']/100
df['Dív.Brut/ Patrim.'] = df['Dív.Brut/ Patrim.']/100
df['Cresc. Rec.5a'] = df['Cresc. Rec.5a']/100
df['Patrim. Líq'] = df['Patrim. Líq']/100

In [10]:
df['prov'] = df['Papel'].str[:4]
df.drop_duplicates(subset='prov', keep='first', inplace=True)
df.drop(columns=['prov'], inplace=True)

In [11]:
# Merging the DataFrames
merged_df = pd.merge(df, df_nome[['Papel', 'Nome Comercial']], on='Papel')
merged_df = pd.merge(merged_df, df_setor[['Papel', 'Setor']], on='Papel')

In [12]:
df_final = merged_df[['Papel', 'Nome Comercial', 'Setor', 'Cotação', 'P/L', 'EV/EBITDA', 'P/VP', 'ROE', 'Div.Yield', 'Cresc. Rec.5a', 'Dív.Brut/ Patrim.', 'Patrim. Líq']]

In [17]:
df = df_final
df = df.rename(columns={'Nome Comercial': 'Empresa'})

In [18]:
# Push each DataFrame to GitHub
push_dataframe_to_github(df, 'df.csv', repository)

Data pushed to df.csv on GitHub successfully!
